In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# File path
data = pd.read_excel(r"INSERT HERE")

# Columns
columns_of_interest = [f"X{i}" for i in range(1, 28)] + ["status", "industry"]
model_data = data[columns_of_interest]

# Industries
industries = ['A', 'C', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'P', 'Q', 'R']

# DataFrame to results
results_df = pd.DataFrame(columns=['Industry', 'Accuracy', 'AUC Score'])
feature_importances = pd.DataFrame()

# individual models
for industry in industries:
    industry_data = model_data[model_data['industry'] == industry]
    
    # Feature & Target
    X = industry_data.drop(['status', 'industry'], axis=1)
    y = industry_data['status']
    
    # Splitting 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=99, stratify=y)

    # RF
    rf_model = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        max_features='log2',
        min_samples_leaf=2,
        min_samples_split=5,
        random_state=99
    )

    # Training
    rf_model.fit(X_train, y_train)

    # Predicting
    y_pred = rf_model.predict(X_test)
    y_proba = rf_model.predict_proba(X_test)[:, 1]

    # Evaluating
    accuracy = accuracy_score(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_proba)

    # result
    new_row = pd.DataFrame({'Industry': [industry], 'Accuracy': [accuracy], 'AUC Score': [auc_score]})
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    
    feature_importances[industry] = rf_model.feature_importances_

# Transpose
feature_importances = feature_importances.T
feature_importances.columns = X.columns

# Print
print("Results Table:")
print(results_df)
print("\nFeature Importance Table:")
print(feature_importances)

# Eksport
results_df.to_excel("model_results_by_industry.xlsx", index=False)
feature_importances.to_excel("feature_importances_by_industry.xlsx")